# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [3]:
# Your code here
# Import necessary packages
import sqlite3
import pandas as pd

# Establish a connection to the database
connection = sqlite3.connect('data.sqlite')


## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [5]:
# Your code here
# Define the SQL query
query = """
SELECT employees.firstName, employees.lastName
FROM employees
JOIN offices ON employees.officeCode = offices.officeCode
WHERE offices.city = 'Boston'
"""

# Execute the query and load the results into a DataFrame
boston_employees_df = pd.read_sql(query, connection)

# Display the DataFrame
print(boston_employees_df)

  firstName   lastName
0     Julie   Firrelli
1     Steve  Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [10]:
# Your code here
# Define the SQL query
query = """
SELECT offices.officeCode, offices.city, COUNT(employees.employeeNumber) AS num_employees
FROM offices
LEFT JOIN employees ON offices.officeCode = employees.officeCode
GROUP BY offices.officeCode, offices.city
HAVING COUNT(employees.employeeNumber) = 0
"""

# Execute the query and load the results into a DataFrame
offices_with_no_employees_df = pd.read_sql(query, connection)

# Display the DataFrame
print(offices_with_no_employees_df)

   officeCode    city  num_employees
0          27  Boston              0


## Write 3 questions of your own and answer them

In [13]:
#Question 1
# How many customers are there per office?
# Define the SQL query
query = """
SELECT offices.officeCode, offices.city, COUNT(customers.customerNumber) AS num_customers
FROM offices
JOIN customers ON offices.officeCode = customers.customerNumber
GROUP BY offices.officeCode, offices.city
"""

# Execute the query and load the results into a DataFrame
customers_per_office_df = pd.read_sql(query, connection)

# Display the DataFrame
print(customers_per_office_df)



Empty DataFrame
Columns: [officeCode, city, num_customers]
Index: []


In [14]:
"""
Question 2: How many orders have been placed by each customer?


"""

# Your code here
query = """
SELECT customers.customerNumber, customers.customerName, COUNT(orders.orderNumber) AS num_orders
FROM customers
JOIN orders ON customers.customerNumber = orders.customerNumber
GROUP BY customers.customerNumber, customers.customerName
"""

# Execute the query and load the results into a DataFrame
orders_per_customer_df = pd.read_sql(query, connection)

# Display the DataFrame
print(orders_per_customer_df)

    customerNumber                    customerName  num_orders
0              103               Atelier graphique           3
1              112              Signal Gift Stores           3
2              114      Australian Collectors, Co.           5
3              119               La Rochelle Gifts           4
4              121              Baane Mini Imports           4
..             ...                             ...         ...
93             486    Motor Mint Distributors Inc.           3
94             487        Signal Collectibles Ltd.           2
95             489  Double Decker Gift Stores, Ltd           2
96             495            Diecast Collectables           2
97             496               Kelly's Gift Shop           4

[98 rows x 3 columns]


In [15]:
"""
Question 3: What is the total sales amount for each product?

"""

# Your code here
query = """
SELECT products.productCode, products.productName, SUM(orderdetails.quantityOrdered * orderdetails.priceEach) AS total_sales
FROM products
JOIN orderdetails ON products.productCode = orderdetails.productCode
GROUP BY products.productCode, products.productName
"""

# Execute the query and load the results into a DataFrame
total_sales_per_product_df = pd.read_sql(query, connection)

# Display the DataFrame
print(total_sales_per_product_df)

    productCode                            productName  total_sales
0      S10_1678  1969 Harley Davidson Ultimate Chopper     90157.77
1      S10_1949               1952 Alpine Renault 1300    190017.96
2      S10_2016                  1996 Moto Guzzi 1100i    109998.82
3      S10_4698   2003 Harley-Davidson Eagle Drag Bike    170686.00
4      S10_4757                    1972 Alfa Romeo GTA    127924.32
..          ...                                    ...          ...
104   S700_3505                            The Titanic     84992.25
105   S700_3962                         The Queen Mary     78919.06
106   S700_4002              American Airlines: MD-11S     71753.93
107    S72_1253                       Boeing X-32A JSF     42692.53
108    S72_3212                             Pont Yacht     47550.40

[109 rows x 3 columns]


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [16]:
# Your code here
query = """
SELECT employees.employeeNumber, employees.firstName, employees.lastName, products.productName
FROM employees
JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders ON customers.customerNumber = orders.customerNumber
JOIN orderdetails ON orders.orderNumber = orderdetails.orderNumber
JOIN products ON orderdetails.productCode = products.productCode
ORDER BY employees.employeeNumber, products.productName
"""

# Execute the query and load the results into a DataFrame
employee_product_sales_df = pd.read_sql(query, connection)

# Display the DataFrame
print(employee_product_sales_df)

      employeeNumber firstName  lastName                          productName
0               1165    Leslie  Jennings  18th Century Vintage Horse Carriage
1               1165    Leslie  Jennings  18th Century Vintage Horse Carriage
2               1165    Leslie  Jennings  18th Century Vintage Horse Carriage
3               1165    Leslie  Jennings  18th Century Vintage Horse Carriage
4               1165    Leslie  Jennings  18th Century Vintage Horse Carriage
...              ...       ...       ...                                  ...
2991            1702    Martin    Gerard                          The Titanic
2992            1702    Martin    Gerard                          The Titanic
2993            1702    Martin    Gerard                          The Titanic
2994            1702    Martin    Gerard            The USS Constitution Ship
2995            1702    Martin    Gerard            The USS Constitution Ship

[2996 rows x 4 columns]


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [17]:
# Your code here
query = """
SELECT employees.employeeNumber, employees.firstName, employees.lastName, SUM(orderdetails.quantityOrdered) AS total_products_sold
FROM employees
JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders ON customers.customerNumber = orders.customerNumber
JOIN orderdetails ON orders.orderNumber = orderdetails.orderNumber
GROUP BY employees.employeeNumber, employees.firstName, employees.lastName
ORDER BY employees.lastName, employees.firstName
"""

# Execute the query and load the results into a DataFrame
products_sold_by_employee_df = pd.read_sql(query, connection)

# Display the DataFrame
print(products_sold_by_employee_df)

    employeeNumber firstName   lastName  total_products_sold
0             1337      Loui     Bondur                 6186
1             1501     Larry       Bott                 8205
2             1401    Pamela   Castillo                 9290
3             1188     Julie   Firrelli                 4227
4             1611      Andy     Fixter                 6246
5             1702    Martin     Gerard                 4180
6             1370    Gerard  Hernandez                14231
7             1165    Leslie   Jennings                11854
8             1504     Barry      Jones                 7486
9             1612     Peter      Marsh                 6632
10            1621      Mami      Nishi                 4923
11            1216     Steve  Patterson                 5561
12            1166    Leslie   Thompson                 4056
13            1286  Foon Yue      Tseng                 5016
14            1323    George     Vanauf                 7423


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [18]:
# Your code here
query = """
SELECT employees.employeeNumber, employees.firstName, employees.lastName, COUNT(DISTINCT products.productCode) AS distinct_products_sold
FROM employees
JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders ON customers.customerNumber = orders.customerNumber
JOIN orderdetails ON orders.orderNumber = orderdetails.orderNumber
JOIN products ON orderdetails.productCode = products.productCode
GROUP BY employees.employeeNumber, employees.firstName, employees.lastName
HAVING COUNT(DISTINCT products.productCode) > 200
ORDER BY employees.lastName, employees.firstName
"""

# Execute the query and load the results into a DataFrame
employees_with_200_products_df = pd.read_sql(query, connection)

# Display the DataFrame
print(employees_with_200_products_df)


Empty DataFrame
Columns: [employeeNumber, firstName, lastName, distinct_products_sold]
Index: []


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!